In [ ]:
# pip install -U autogen-agentchat autogen-ext[openai,web-surfer]
# playwright install
#%pip install python.dotenv

## Example using the ROUND ROBING GROUPCHAT team with a USER PROXY AGENT and a MULTIMODAL WEBSURFER

In [3]:

import asyncio
from autogen_agentchat.agents import UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_ext.agents.web_surfer import MultimodalWebSurfer
from dotenv import load_dotenv

from settings import llm_config
load_dotenv()

True

In [4]:
model_client = AzureOpenAIChatCompletionClient.load_component(llm_config)

In [5]:
async def main() -> None:
    # The web surfer will open a Chromium browser window to perform web browsing tasks.
    web_surfer = MultimodalWebSurfer("web_surfer", model_client, headless=False, animate_actions=True)
    # The user proxy agent is used to get user input after each step of the web surfer.
    # NOTE: you can skip input by pressing Enter.
    user_proxy = UserProxyAgent("user_proxy")
    # The termination condition is set to end the conversation when the user types 'exit'.
    termination = TextMentionTermination("exit", sources=["user_proxy"])
    # Web surfer and user proxy take turns in a round-robin fashion.
    team = RoundRobinGroupChat([web_surfer, user_proxy], termination_condition=termination)
    try:
        # Start the team and wait for it to terminate.
        await Console(team.run_stream(task="find information about the latest iPhone"))
    finally:
        await web_surfer.close()
        await model_client.close()

await main()

---------- user ----------
find information about the latest iPhone
---------- web_surfer ----------
Web surfing error:

Traceback (most recent call last):
  File "/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/httpx/_transports/default.py", line 394, in handle_async_request
    resp = await self._pool.handle_async_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/httpcore/_async/connection_pool.py", line 207, in handle_async_request
    raise UnsupportedProtocol(
        "Request URL is missing an 'http://' or 'https://' protocol."
    )
httpcore.UnsupportedProtocol: Request URL is missing an 'http://' or 'https://' protocol.

The above exception was the direct cause of the following exception:

Traceback (most r